In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import librosa 

plt.rcParams.update({'font.size': 12});

In [ ]:
#Load audio file
filename = librosa.util.example_audio_file()
y, sr = librosa.load(filename)
y, _ = librosa.effects.trim(y, top_db=20)

#setup figure
fig = plt.figure(dpi=200)
ax2 = fig.add_subplot(1,1,1)
ax2.set_title('Relevant time dependencies for sound generation')

max_scale = 2**18+80000
repeat_length = int(14*8)*2

ax2.set_xlim([0, max_scale])
ax2.set_ylim([y.min()*1.1, y.max()*1.1])

#set up list of images for animation

im2, = ax2.plot([], [], color=(51/255,51/255,1))
im3, = ax2.plot([], [], color=(1,51/255,51/255))

def func(n):
    n = n + 96
    n = n/2
    start = int(max_scale-2**(n/8))
    end =  int(max_scale+2**(n/8))
    
    inner_n = ((n-2)//32)*32
    inner_start = int(max_scale-2**(inner_n/8))
    inner_end = int(max_scale+2**(inner_n/8))
    
    im2.set_data(np.arange(start, end), y[start:end])           
    im3.set_data(np.arange(inner_start, inner_end), 
                 y[inner_start:inner_end])       
    
    lim = ax2.set_xlim(start, end)
    ax2.set_xticks([start, start+(end-start)/2, end])
    
    half_length = (end-start)/2
    if half_length/sr>2:  
        ax2.set_xlabel('seconds')
        ax2.set_xticklabels([str(int(-half_length/sr)), 0,  str(int(half_length/sr))])
    elif half_length/sr*1e3>10:  
        ax2.set_xlabel('milliseconds')
        ax2.set_xticklabels([str(int(-half_length/sr*1e3)), 0,  str(int(half_length/sr*1e3))])
    elif half_length/sr*1e6>1:  
        ax2.set_xlabel('microseconds')
        ax2.set_xticklabels([str(int(-half_length/sr*1e6)), 0,  str(int(half_length/sr*1e6))])
    
    return im2

ani = animation.FuncAnimation(fig, func, frames=repeat_length, interval=100, blit=False, repeat=True)

plt.show()

ani.save('time_signal_zoom_out.mp4')

In [ ]:
#compute spectrogram

from tifresi.stft import GaussTruncTF
from tifresi.transforms import log_spectrogram
from tifresi.utils import preprocess_signal

dynamic_range_dB = 50

anStft = GaussTruncTF(hop_size=256, stft_channels=1024)

spectrogram = anStft.spectrogram(y)
logSpectrogram = log_spectrogram(spectrogram, dynamic_range_dB=dynamic_range_dB)

logSpectrogram = logSpectrogram / (dynamic_range_dB / 2) + 1



In [ ]:
logSpectrogram.shape

In [ ]:
fig = plt.figure(figsize = (12, 4))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

start = 1024
end = start+8096

ax1.plot(np.arange(start, end), y[start:end])
ax2.imshow(logSpectrogram[:, start//256:end//256], interpolation='nearest', cmap='inferno', aspect='auto')
ax2.invert_yaxis()
ax1.set(xlabel="Time", ylabel="Amplitude", title='Time signal')
ax2.set(xlabel="Time", ylabel="Frequency", title='Spectrogram')

plt.show()

plt.savefig("time-to-spectrogram.png", 
               bbox_inches='tight', dpi=200)

In [ ]:
#setup figure
fig = plt.figure(dpi=200)
ax2 = fig.add_subplot(1,1,1)

max_scale = logSpectrogram.shape[1]//2
width=0

#set up list of images for animation

im2 = ax2.imshow(np.flip(logSpectrogram), interpolation='nearest', cmap='inferno', aspect='auto')

im3, = ax2.plot([], [], 'b--', lw=2)
im4, = ax2.plot([], [], 'b--', lw=2)

lim = ax2.set_xlim(max_scale-1, max_scale+1)
ax2.tick_params(
        axis='y',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off,
        left=False,
        top=False,         # ticks along the top edge are off
        labelleft=False,
        labelbottom=False) # labels along the bottom edge are off

def func(n):
    n = n + 6
    width = int(2**(n/8))
    inner_width = int(2**(((n-1)//32)*32/8))
    
    im3.set_data([max_scale-inner_width, max_scale-inner_width], [0, 512])
    im4.set_data([max_scale+inner_width, max_scale+inner_width], [0, 512])
    
    lim = ax2.set_xlim(max_scale-width, max_scale+width)
    ax2.set_xticks([max_scale-width, max_scale, max_scale+width])

    
    if width*256/sr>2:  
        ax2.set_xlabel('seconds')
        ax2.set_xticklabels([str(int(-width*256/sr)), 0,  str(int(width*256/sr))])
    else:  
        ax2.set_xlabel('milliseconds')
        ax2.set_xticklabels([str(int(-width*256/sr*1e3)), 0,  str(int(width*256/sr*1e3))])
    
    return im2

ani = animation.FuncAnimation(fig, func, frames=84, interval=100, blit=False, repeat=False)

plt.show()

ani.save('spectrogram_zoom_out.mp4')